In [ ]:
import pandas as pd
import numpy as np
import joblib
import warnings
from IPython.display import display

warnings.filterwarnings('ignore')

print("=======================================================")
print("   🤖 QUANT ENGINE v12 (ADAPTIVE DRAW BIAS)            ")
print("=======================================================")

# ==============================================================================
# 1. CONFIGURATION
# ==============================================================================
BANKROLL = 30.00        
MIN_EDGE = 0.01           
MAX_ODDS_CAP = 6.00       
MAX_STAKE_PCT = 0.10      
KELLY_FRACTION = 0.25     

# --- THE TUNING KNOB ---
# How much to boost Draw Probability in tight games.
# 0.08 (8%) is strong enough to flip Burnley, but weak enough to keep Villa.
DRAW_BIAS = 0.10  

# --- FIXTURES ---
fixtures = [
    {"Home": "Manchester United", "Away": "Newcastle United", "Odds_1": 2.53, "Odds_X": 3.61, "Odds_2": 2.68},
    {"Home": "Nottingham Forest", "Away": "Manchester City", "Odds_1": 5.29, "Odds_X": 4.45, "Odds_2": 1.58},
    {"Home": "Arsenal",           "Away": "Brighton & Hove Albion", "Odds_1": 1.40, "Odds_X": 4.92, "Odds_2": 7.85},
    {"Home": "Brentford",         "Away": "Bournemouth",      "Odds_1": 2.29, "Odds_X": 3.60, "Odds_2": 3.03},
    {"Home": "Burnley",           "Away": "Everton",          "Odds_1": 4.01, "Odds_X": 3.35, "Odds_2": 2.01},
    {"Home": "Liverpool",         "Away": "Wolverhampton Wanderers", "Odds_1": 1.24, "Odds_X": 6.59, "Odds_2": 11.28},
    {"Home": "West Ham United",   "Away": "Fulham",           "Odds_1": 2.68, "Odds_X": 3.41, "Odds_2": 2.63},
    {"Home": "Chelsea",           "Away": "Aston Villa",      "Odds_1": 1.85, "Odds_X": 3.88, "Odds_2": 4.03},
    {"Home": "Sunderland",        "Away": "Leeds United",     "Odds_1": 2.58, "Odds_X": 3.23, "Odds_2": 2.86},
    {"Home": "Crystal Palace",    "Away": "Tottenham Hotspur", "Odds_1": 2.29, "Odds_X": 3.30, "Odds_2": 3.25},
]

# ==============================================================================
# 2. LOAD MODEL
# ==============================================================================
try:
    artifacts = joblib.load('football_model_final.pkl')
    model = artifacts['model']
    features = artifacts['features']
    current_elos = artifacts['elo_dict']
    df_recent = artifacts['df_recent']
    print(f"✅ Model Loaded. Bias Strength: {DRAW_BIAS*100}%")
except:
    print("❌ Error loading pickle.")
    exit()

def get_stats(team):
    rows = df_recent[(df_recent['home_team_name'] == team) | (df_recent['away_team_name'] == team)]
    if len(rows) == 0: return None
    last = rows.sort_values('date').iloc[-1]
    prefix = 'home_' if last['home_team_name'] == team else 'away_'
    stats = {}
    roll_feats = ['team_xg', 'team_possession', 'shots_onTarget', 'corners', 'team_points', 'fouls', 'team_score']
    for f in roll_feats:
        col = f"{prefix}roll_{f}"
        stats[f] = last[col] if col in last else 0
    return stats

# ==============================================================================
# 3. ANALYSIS LOOP
# ==============================================================================
opportunities = []
ELITE_TEAMS = ["Manchester City", "Liverpool", "Arsenal"]

print(f"\n🔍 ANALYZING {len(fixtures)} MATCHES...")

for f in fixtures:
    h_team, a_team = f['Home'], f['Away']
    h_stats, a_stats = get_stats(h_team), get_stats(a_team)
    h_elo, a_elo = current_elos.get(h_team, 1500), current_elos.get(a_team, 1500)
    
    if not h_stats or not a_stats: continue

    input_data = {
        'diff_elo': (h_elo + 70) - a_elo, 
        'home_elo': h_elo, 'away_elo': a_elo, 'diff_rest': 0, 
        'home_roll_team_xg': h_stats['team_xg'], 'away_roll_team_xg': a_stats['team_xg'],
        'home_roll_team_possession': h_stats['team_possession'], 'away_roll_team_possession': a_stats['team_possession'],
        'home_roll_shots_onTarget': h_stats['shots_onTarget'], 'away_roll_shots_onTarget': a_stats['shots_onTarget'],
        'home_roll_corners': h_stats['corners'], 'away_roll_corners': a_stats['corners'],
        'home_roll_fouls': h_stats['fouls'], 'away_roll_fouls': a_stats['fouls']
    }
    
    roll_feats = ['team_xg', 'team_possession', 'shots_onTarget', 'corners', 'team_points', 'fouls']
    for feat in roll_feats:
        input_data[f"diff_{feat}"] = h_stats[feat] - a_stats[feat]

    # Predict Raw
    input_df = pd.DataFrame([input_data]).reindex(columns=features, fill_value=0)
    probs = model.predict_proba(input_df)[0]
    p_away, p_draw, p_home = probs[0], probs[1], probs[2]

    # --- 1. DEADLOCK DETECTION & ADJUSTMENT ---
    # Logic: If game is tight (<12% gap), shifts probs to favor draw
    is_tight = abs(p_home - p_away) < 0.12
    reason_tag = "Standard"
    
    # Store Adjusted Probabilities for Calculation
    adj_p_home, adj_p_draw, adj_p_away = p_home, p_draw, p_away

    if is_tight:
        # APPLY THE BIAS
        adj_p_draw += DRAW_BIAS
        adj_p_home -= (DRAW_BIAS / 2)
        adj_p_away -= (DRAW_BIAS / 2)
        reason_tag = "Deadlock (Draw Boosted)"

    # --- 2. GOLIATH FILTER ---
    # Don't let adjusting probs trick us into betting against City
    block_underdog = False
    if (a_team in ELITE_TEAMS and h_elo < a_elo - 200) or (h_team in ELITE_TEAMS and a_elo < h_elo - 200):
        block_underdog = True
        reason_tag = "Goliath Match"

    print(f"{h_team[:10]} vs {a_team[:10]}: H {p_home:.2f} | D {p_draw:.2f} | A {p_away:.2f} -> {reason_tag}")

    # --- 3. VALUE FINDER (Using ADJUSTED Probabilities) ---
    bets_for_match = []
    
    for outcome, p, odds, label in [
        ('HOME', adj_p_home, f['Odds_1'], h_team),
        ('DRAW', adj_p_draw, f['Odds_X'], 'Draw'),
        ('AWAY', adj_p_away, f['Odds_2'], a_team)
    ]:
        if odds > MAX_ODDS_CAP: continue
        
        # Apply Goliath Safety
        if block_underdog:
            # Only allow betting if this outcome was the ORIGINAL favorite
            raw_p = p_home if outcome == 'HOME' else (p_away if outcome == 'AWAY' else p_draw)
            if raw_p != max(p_home, p_draw, p_away): continue 

        edge = (p * odds) - 1
        
        # Min Probability (Adjusted)
        min_p = 0.23 if outcome == 'DRAW' else 0.20
        
        if edge > MIN_EDGE and p > min_p:
            bets_for_match.append({
                'Match': f"{h_team} vs {a_team}",
                'Type': outcome,
                'Team': label,
                'Odds': odds,
                'Prob': p,     # Using Adjusted Prob for Kelly
                'Edge': edge,  # Using Adjusted Edge for Sorting
                'Reason': reason_tag
            })

    # --- 4. SELECTION (PURE EDGE COMPARISON) ---
    if bets_for_match:
        # We simply pick the bet with the Highest Adjusted Edge.
        # This handles the "Villa vs Chelsea" logic naturally.
        # If Villa Edge (even after penalty) > Draw Edge (after boost), Villa wins.
        best = sorted(bets_for_match, key=lambda x: x['Edge'], reverse=True)[0]
        opportunities.append(best)

# ==============================================================================
# 4. PORTFOLIO
# ==============================================================================
if opportunities:
    res = pd.DataFrame(opportunities)
    
    def calculate_stake(row):
        b = row['Odds'] - 1
        p = row['Prob']
        q = 1 - p
        f = ((b * p) - q) / b
        return max(0, min(f * KELLY_FRACTION, MAX_STAKE_PCT))

    res['Stake_Pct'] = res.apply(calculate_stake, axis=1)
    res['Stake'] = res['Stake_Pct'] * BANKROLL
    res['Profit'] = res['Stake'] * (res['Odds'] - 1)
    
    res = res[res['Stake'] > 0.01].sort_values('Stake', ascending=False)
    
    display_df = res[['Match', 'Type', 'Team', 'Odds', 'Prob', 'Edge', 'Reason', 'Stake', 'Profit']].copy()
    display_df.columns = ['Match', 'Bet Type', 'Selection', 'Odds', 'Adj Win %', 'Adj Edge', 'Logic', 'Stake ($)', 'Profit ($)']
    
    styled = (display_df.style
        .format({'Odds': '{:.2f}', 'Adj Win %': '{:.1%}', 'Adj Edge': '{:.1%}', 'Stake ($)': '${:.2f}', 'Profit ($)': '${:.2f}'})
        .background_gradient(subset=['Adj Win %'], cmap='Greens', vmin=0.2, vmax=0.8)
        .background_gradient(subset=['Adj Edge'], cmap='Blues', vmin=0.0, vmax=0.5)
        .bar(subset=['Stake ($)'], color='#ff6b6b', vmin=0)
        .bar(subset=['Profit ($)'], color='#51cf66', vmin=0)
        .set_caption(f"🚀 FINAL QUANT SLIP v12 (Adaptive Bias) | Bias: {DRAW_BIAS*100}%")
    )
    display(styled)
    
    print(f"\n💰 Total Risk: ${res['Stake'].sum():.2f}")
else:
    print("\n📉 No Value Bets Found.")

   🤖 HYBRID QUANT ENGINE (CONFIDENCE + VALUE)         
✅ Model Loaded. Features: 18

🔍 ANALYZING 10 MATCHES...
Manchester United vs Newcastle United: H 0.55 | D 0.17 | A 0.28
Nottingham Forest vs Manchester City: H 0.30 | D 0.20 | A 0.50
Arsenal vs Brighton & Hove Albion: H 0.67 | D 0.19 | A 0.14
Brentford vs Bournemouth: H 0.42 | D 0.24 | A 0.33
Burnley vs Everton: H 0.37 | D 0.22 | A 0.42
Liverpool vs Wolverhampton Wanderers: H 0.75 | D 0.13 | A 0.12
West Ham United vs Fulham: H 0.43 | D 0.23 | A 0.34
Chelsea vs Aston Villa: H 0.38 | D 0.23 | A 0.39
Sunderland vs Leeds United: H 0.42 | D 0.34 | A 0.24
Crystal Palace vs Tottenham Hotspur: H 0.57 | D 0.24 | A 0.19


,Match,Bet Type,Selection,Odds,Win %,Edge,Stake ($),Profit ($)
0,Manchester United vs Newcastle United,HOME,Manchester United,2.53,55.2%,39.6%,$1.94,$2.97
6,Crystal Palace vs Tottenham Hotspur,HOME,Crystal Palace,2.29,56.9%,30.3%,$1.76,$2.28
4,Chelsea vs Aston Villa,AWAY,Aston Villa,4.03,38.8%,56.2%,$1.39,$4.22
2,Burnley vs Everton,HOME,Burnley,4.01,36.6%,46.6%,$1.16,$3.50
1,Nottingham Forest vs Manchester City,HOME,Nottingham Forest,5.29,30.0%,58.6%,$1.02,$4.39
3,West Ham United vs Fulham,HOME,West Ham United,2.68,43.2%,15.9%,$0.71,$1.19
5,Sunderland vs Leeds United,HOME,Sunderland,2.58,42.4%,9.5%,$0.45,$0.71



💰 Total Risk: $8.44
💵 Remaining:  $21.56
